### Import Packages

In [1]:
import os
import pyodbc
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

### Import Data

#### `IM_I_RESULTSRELEASED_S.dbo.PortfolioAbsReturnReleased`

In [2]:
cnxn = pyodbc.connect('Driver={SQL Server Native Client 11.0};\
                       Server=lasr-sqldb-prd-im,17001;\
                       Database=IM_S_FAMIS_S;\
                       Trusted_Connection=yes;')

In [3]:
sql_query = """

select
A.*
from
(
select
AccountUID,
ShareClass,
AsOfDate as POSTDT,
PeriodType as periodtype,
FactType,
VehicleType,
VehicleAbbrevName,
TotalReturn as rtnpct,
FileTimeStamp
--BenchmarkID
from IM_S_FAMIS_S.dbo.famis_gcsima_portfolio_performance_f1_v1_t1
where TotalReturn is not NULL AND AccountUID is not NULL
) as A
  inner join
(
select
AccountUID,
ShareClass,
AsOfDate as POSTDT,
PeriodType as periodtype,
FactType,
VehicleType,
VehicleAbbrevName,
--BenchmarkID
max(FileTimeStamp) as max_time
from IM_S_FAMIS_S.dbo.famis_gcsima_portfolio_performance_f1_v1_t1
where TotalReturn is not NULL AND AccountUID is not NULL
group by AccountUID,ShareClass,AsOfDate,PeriodType,FactType,VehicleType,VehicleAbbrevName
) as B
on A.AccountUID=B.AccountUID and A.ShareClass=B.ShareClass and A.POSTDT=B.POSTDT and A.periodtype=B.periodtype and A.FactType=B.FactType 
and A.VehicleType=B.VehicleType and A.VehicleAbbrevName=B.VehicleAbbrevName and A.FileTimeStamp=B.max_time
"""

# Save the data 
data = pd.read_sql(sql_query, cnxn)

# Close the connection with LASR
cnxn.close()

(1787446, 9)


,AccountUID,ShareClass,POSTDT,periodtype,FactType,VehicleType,VehicleAbbrevName,rtnpct,FileTimeStamp
0,100063,529A,2012-10-31,1MTD,TRNAV,American Fund,CF24,99.700000,2017-08-14 19:01:34
1,176100239,M,2017-11-30,1MTD,TRNAV,107,CAPCM,99.392161,2017-12-14 23:36:03
2,1101337,CAD-A,2007-06-30,6MTD,TRNAV,CIAM Fund,CGCFE,106.870000,2017-06-14 13:21:02
3,10100073,1,2014-08-31,LTD,TRNAV,AFIS Fund,VICIB,102.410000,2017-08-14 19:01:34
4,176100239,M,2017-12-31,1MTD,TRNAV,PCS Fund,CAPCM,100.364214,2018-01-16 23:36:03


In [4]:
print(data.shape)
data.head()

(1787446, 9)


,AccountUID,ShareClass,POSTDT,periodtype,FactType,VehicleType,VehicleAbbrevName,rtnpct,FileTimeStamp
0,100063,529A,2012-10-31,1MTD,TRNAV,American Fund,CF24,99.700000,2017-08-14 19:01:34
1,176100239,M,2017-11-30,1MTD,TRNAV,107,CAPCM,99.392161,2017-12-14 23:36:03
2,1101337,CAD-A,2007-06-30,6MTD,TRNAV,CIAM Fund,CGCFE,106.870000,2017-06-14 13:21:02
3,10100073,1,2014-08-31,LTD,TRNAV,AFIS Fund,VICIB,102.410000,2017-08-14 19:01:34
4,176100239,M,2017-12-31,1MTD,TRNAV,PCS Fund,CAPCM,100.364214,2018-01-16 23:36:03


In [5]:
data['postdt_new'] = data['POSTDT'].apply(lambda dt: dt.replace(day=1))
data.head()

,AccountUID,ShareClass,POSTDT,periodtype,FactType,VehicleType,VehicleAbbrevName,rtnpct,FileTimeStamp,postdt_new
0,100063,529A,2012-10-31,1MTD,TRNAV,American Fund,CF24,99.700000,2017-08-14 19:01:34,2012-10-01
1,176100239,M,2017-11-30,1MTD,TRNAV,107,CAPCM,99.392161,2017-12-14 23:36:03,2017-11-01
2,1101337,CAD-A,2007-06-30,6MTD,TRNAV,CIAM Fund,CGCFE,106.870000,2017-06-14 13:21:02,2007-06-01
3,10100073,1,2014-08-31,LTD,TRNAV,AFIS Fund,VICIB,102.410000,2017-08-14 19:01:34,2014-08-01
4,176100239,M,2017-12-31,1MTD,TRNAV,PCS Fund,CAPCM,100.364214,2018-01-16 23:36:03,2017-12-01


In [7]:
#check no duplicate records
print(data.drop_duplicates(subset=['AccountUID', 'ShareClass', 'postdt_new', 'periodtype', 'FactType'], keep='first', inplace=False).count())

AccountUID           1787446
ShareClass           1787446
POSTDT               1787446
periodtype           1787446
FactType             1787446
VehicleType          1787446
VehicleAbbrevName    1787446
rtnpct               1787446
FileTimeStamp        1787446
postdt_new           1787446
dtype: int64


In [14]:
df_lvl1 = data.groupby(['postdt_new', 'periodtype', 'FactType', 'VehicleType', 'VehicleAbbrevName'])['AccountUID', 'ShareClass'].count().reset_index()
df_lvl2 = data.groupby(['ShareClass','postdt_new', 'periodtype', 'FactType', 'VehicleType', 'VehicleAbbrevName'])['AccountUID'].count().reset_index()

In [15]:
print(df_lvl1.shape)
df_lvl1.head()

(176027, 7)


,postdt_new,periodtype,FactType,VehicleType,VehicleAbbrevName,AccountUID,ShareClass
0,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIAA,4,4
1,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIBF,4,4
2,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIG,5,5
3,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGI,5,5
4,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGL,4,4


In [16]:
print(df_lvl2.shape)
df_lvl2.head()

(1787446, 7)


,ShareClass,postdt_new,periodtype,FactType,VehicleType,VehicleAbbrevName,AccountUID
0,1,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIBF,1
1,1,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIG,1
2,1,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGI,1
3,1,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGL,1
4,1,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGVT,1


In [25]:
df_lvl1.rename(columns={'AccountUID':'Account_Cnt'}, inplace=True)
df_lvl1.drop(['ShareClass'], axis=1, inplace=True)
df_lvl1.head()

,postdt_new,periodtype,FactType,VehicleType,VehicleAbbrevName,Account_Cnt
0,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIAA,4
1,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIBF,4
2,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIG,5
3,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGI,5
4,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGL,4


In [26]:
df_lvl2.rename(columns={'AccountUID':'AccSC_Cnt'}, inplace=True)
df_lvl2.drop(['ShareClass'], axis=1, inplace=True)
df_lvl2.head()

,postdt_new,periodtype,FactType,VehicleType,VehicleAbbrevName,AccSC_Cnt
0,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIBF,1
1,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIG,1
2,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGI,1
3,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGL,1
4,2007-06-01,10YTD,ARRNAV,AFIS Fund,VIGVT,1


In [27]:
df_lvl1.to_csv('summary_FAMIS_AccountLevel.csv')
df_lvl2.to_csv('summary_FAMIS_AccSCLevel.csv')